# Chapter 10-03: ZeRO Redundancy Optimizer

## 학습 목표
- DP 학습에서 각 GPU가 **동일한 파라미터, 그래디언트, 옵티마이저 상태**를 중복 보유하는 구조적 낭비를 파악한다
- ZeRO Stage 1 → 2 → 3의 파티셔닝 단계별 메모리 절약 공식 $M \propto 1/N$을 수식으로 증명한다
- 각 Stage에서 추가되는 **통신 비용 패턴**을 분석해 Stage 선택 기준을 제시한다
- NumPy 시뮬레이션으로 ZeRO Stage 1~3의 파티셔닝과 통신 과정을 직접 구현한다

## 목차
1. [수학적 기초: ZeRO 단계별 메모리 공식](#1.-수학적-기초)
2. [ZeRO Stage 1: Optimizer State 분산](#3.-Stage-1)
3. [ZeRO Stage 2: Gradient 분산](#4.-Stage-2)
4. [ZeRO Stage 3: Parameter 분산](#5.-Stage-3)
5. [단계별 메모리 절약 시뮬레이션](#6.-시뮬레이션)
6. [Stage 선택 가이드 및 종합 비교](#7.-Stage-선택)
7. [정리 및 연습 문제](#8.-정리)

## 1. 수학적 기초

### ZeRO 단계별 메모리 절약 수식

Mixed Precision + Adam 기준 GPU당 메모리 ($N$ = GPU 수):

| ZeRO Stage | 분산 대상 | GPU당 메모리 | 비율 |
|------------|----------|------------|------|
| **Baseline (DP)** | 없음 | $16P$ bytes | 100% |
| **Stage 1** | Optimizer States ($m$, $v$) | $4P + \frac{8P}{N}$ bytes | ~50% (N→∞) |
| **Stage 2** | Optimizer States + Gradients | $2P + \frac{6P}{N}$ bytes | ~12.5% (N→∞) |
| **Stage 3** | Optimizer States + Grads + Params | $\frac{16P}{N}$ bytes | $\frac{1}{N}$ |

### Stage 3 유도 과정

각 GPU가 전체 파라미터의 $1/N$만 항상 보유:

$$M_{Stage3} = \frac{2P}{N} + \frac{2P}{N} + \frac{4P}{N} + \frac{4P}{N} + \frac{4P}{N} = \frac{16P}{N}$$

→ **N배 메모리 절약**, 단 Forward/Backward 시 **파라미터 All-Gather** 필요

### 통신 비용 추가

$$\Delta T_{Stage3} = 3 \times T_{AllReduce}(K)$$

(Forward 1회 All-Gather + Backward 1회 All-Gather + Grad Reduce-Scatter 1회)

---

### 🐣 초등학생을 위한 친절 설명!

#### 😕 왜 GPU마다 같은 데이터가 중복되나요?

> 반 학생 8명이 같은 교과서 공부를 하는데,  
> 각자 교과서 1권씩, 노트(그래디언트) 1권씩, 성적록(Adam 상태) 2권씩 갖고 있어요.
> (총 4가지 × 8명 = 32개의 중복!)
>
> **ZeRO의 아이디어**: 이걸 나눠서 가지면 됩니다!
> - **Stage 1**: 성적록(Adam)만 1/8씩 나눠 가짐
> - **Stage 2**: 성적록 + 노트도 1/8씩 나눠 가짐
> - **Stage 3**: 교과서까지 1/8씩 나눠 가짐 (필요할 때만 서로 빌림)

### 📝 연습 문제

#### 문제 1: ZeRO Stage 3 메모리 계산

Llama-2 70B 모델, N=64 GPU, ZeRO Stage 3 적용 시  
GPU당 필요 메모리는? (Mixed Precision + Adam)

<details>
<summary>💡 풀이 확인</summary>

$P = 70$B, $N = 64$

$$M_{Stage3} = \frac{16 \times 70}{64} = \frac{1120}{64} = \mathbf{17.5 \text{ GB}}$$

단일 H100 80GB에서 충분히 수용 가능! (Activation 추가 필요)

비교: Baseline = 16 × 70 = 1120 GB → 64대 없으면 불가
</details>

#### 문제 2: Stage 선택 기준

GPU 8대에서 7B 모델 학습 시, 어떤 Stage가 적절한가?  
각 Stage의 GPU당 메모리를 계산하고 H100 80GB 기준으로 판단하라.

<details>
<summary>💡 풀이 확인</summary>

$P = 7$B, $N = 8$, $(16P = 112$ GB)

- **Baseline**: 112 GB → H100 초과 ❌
- **Stage 1**: $4×7 + 8×7/8 = 28 + 7 = 35$ GB → H100 OK ✅ (여유 있음)
- **Stage 2**: $2×7 + 6×7/8 = 14 + 5.25 = 19.25$ GB → 더 여유
- **Stage 3**: $112/8 = 14$ GB → 최소 메모리

7B, 8GPU에서는 **Stage 1**도 충분 (통신 오버헤드 최소화 우선)
</details>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(42)

# ---------------------------------------------------
# ZeRO 단계별 메모리 공식 구현
# ---------------------------------------------------

def memory_baseline(P_B, N):
    """Baseline DP: 각 GPU가 전체 복사본 보유 = 16P"""
    return 16 * P_B

def memory_zero1(P_B, N):
    """ZeRO Stage 1: Optimizer State 분산 = 4P + 8P/N"""
    return 4 * P_B + 8 * P_B / N

def memory_zero2(P_B, N):
    """ZeRO Stage 2: +Gradient 분산 = 2P + 6P/N"""
    return 2 * P_B + 6 * P_B / N

def memory_zero3(P_B, N):
    """ZeRO Stage 3: +Param 분산 = 16P/N"""
    return 16 * P_B / N

# 7B 모델 기준 단계별 메모리
P = 7  # 7B params
N_range = [1, 2, 4, 8, 16, 32, 64, 128]

print(f"Llama-2 7B 모델 (P={P}B, Mixed Precision + Adam)")
print(f"\n{'N (GPU)':>10} | {'Baseline':>10} | {'Stage 1':>10} | {'Stage 2':>10} | {'Stage 3':>10}")
print("-" * 58)
for N in N_range:
    b  = memory_baseline(P, N)
    s1 = memory_zero1(P, N)
    s2 = memory_zero2(P, N)
    s3 = memory_zero3(P, N)
    h100 = 80 * 0.7  # H100 70% 사용 기준
    def fits(m): return '✅' if m <= h100 else '❌'
    print(f"{N:>10} | {b:>8.1f}GB{fits(b)} | {s1:>8.1f}GB{fits(s1)} | {s2:>8.1f}GB{fits(s2)} | {s3:>8.1f}GB{fits(s3)}")

## 3 ~ 5. Stage 1/2/3 시뮬레이션

In [ ]:
# ---------------------------------------------------
# ZeRO Stage 1~3 파티셔닝 및 통신 패턴 시뮬레이션
# ---------------------------------------------------

class ZeROSimulator:
    """
    ZeRO Stage 1~3 동작을 NumPy로 시뮬레이션.
    각 Stage에서 파티셔닝과 통신 패턴을 확인.
    """

    def __init__(self, n_params, n_gpus, stage=3):
        self.P = n_params
        self.N = n_gpus
        self.stage = stage
        self.chunk = n_params // n_gpus
        assert n_params % n_gpus == 0

        # 전체 파라미터/그래디언트/옵티마이저 초기화
        self.params = np.random.randn(n_params).astype(np.float32)
        self.optim_m = np.zeros(n_params, dtype=np.float32)  # Adam m
        self.optim_v = np.zeros(n_params, dtype=np.float32)  # Adam v
        self.communication_log = []

    def _slice(self, rank):
        return slice(rank * self.chunk, (rank + 1) * self.chunk)

    def memory_per_gpu(self):
        """현재 Stage의 이론 GPU당 메모리 (파라미터 단위)"""
        if self.stage == 0:
            return 16  # Baseline: 16×P_count bytes
        elif self.stage == 1:
            return 4 + 8 / self.N  # param FP16 + grad FP16 + master FP32 + optim/N
        elif self.stage == 2:
            return 2 + 6 / self.N  # param FP16 + (grad+master+optim)/N
        else:
            return 16 / self.N

    def forward_pass(self, x):
        """Forward: Stage 3에서는 All-Gather로 파라미터 수집"""
        if self.stage == 3:
            # 각 GPU가 자신 파티션만 보유 → All-Gather 필요
            self.communication_log.append(f'[Forward] All-Gather params ({self.P} elements)')
            full_params = self.params  # 시뮬레이션 상 전체 접근
        else:
            full_params = self.params
        return x @ full_params.reshape(-1, 1) if x.ndim == 2 else x * full_params[:len(x)]

    def backward_pass(self, x, grad_output):
        """Backward: Stage 2,3에서는 Reduce-Scatter로 그래디언트 분산"""
        raw_grad = np.random.randn(self.P).astype(np.float32) * 0.01  # 임의 그래디언트
        if self.stage >= 2:
            self.communication_log.append(f'[Backward] Reduce-Scatter gradients → 각 GPU가 청크만 보유')
        else:
            self.communication_log.append(f'[Backward] All-Reduce gradients (전체 복사본 유지)')
        return raw_grad

    def optimizer_step(self, grads, rank=0, lr=1e-3):
        """옵티마이저 업데이트: 각 GPU가 자기 파티션만 업데이트"""
        sl = self._slice(rank) if self.stage >= 1 else slice(None)
        self.optim_m[sl] = 0.9 * self.optim_m[sl] + 0.1 * grads[sl]
        self.optim_v[sl] = 0.99 * self.optim_v[sl] + 0.01 * grads[sl] ** 2
        m_hat = self.optim_m[sl] / (1 - 0.9)
        v_hat = self.optim_v[sl] / (1 - 0.99)
        self.params[sl] -= lr * m_hat / (np.sqrt(v_hat) + 1e-8)

        if self.stage == 3:
            self.communication_log.append(f'[OptimStep] All-Gather updated params → 전파')

    def simulate_step(self, x, rank=0):
        """1 학습 스텝 시뮬레이션"""
        self.communication_log = []
        out = self.forward_pass(x)
        grads = self.backward_pass(x, 1.0)
        self.optimizer_step(grads, rank=rank)
        return self.communication_log


# 시뮬레이션 실행
n_params = 16
n_gpus   = 4
x_input  = np.ones((1, n_params))

for stage in [0, 1, 2, 3]:
    name = {0: 'Baseline (DP)', 1: 'ZeRO Stage 1', 2: 'ZeRO Stage 2', 3: 'ZeRO Stage 3'}[stage]
    sim = ZeROSimulator(n_params, n_gpus, stage=stage)
    comm = sim.simulate_step(x_input, rank=0)
    mem_ratio = sim.memory_per_gpu()
    print(f"\n{'─'*55}")
    print(f"  {name}")
    print(f"  GPU당 메모리: {mem_ratio:.2f} × P_count 파라미터 단위")
    print(f"  통신 패턴:")
    for c in comm:
        print(f"    • {c}")

## 6. 시뮬레이션

In [ ]:
# ---------------------------------------------------
# ZeRO Stage별 메모리 절약 종합 시각화
# ---------------------------------------------------

models = [
    ('7B',  7),  ('13B', 13), ('70B', 70), ('175B', 175)
]
N = 64
gpu_limit = 80 * 0.7  # H100 70% 활용

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# ── 왼쪽: 모델별 Stage 메모리 비교 ──────────────────
ax1 = axes[0]
x = np.arange(len(models))
w = 0.18
for i, (stage_fn, label, color) in enumerate([
    (memory_baseline, 'Baseline', '#E53935'),
    (memory_zero1, 'Stage 1', '#FB8C00'),
    (memory_zero2, 'Stage 2', '#43A047'),
    (memory_zero3, 'Stage 3', '#1E88E5'),
]):
    vals = [stage_fn(p, N) for _, p in models]
    bars = ax1.bar(x + i*w, vals, w, label=label, color=color, alpha=0.85)

ax1.axhline(y=gpu_limit, color='k', ls='--', lw=2, label=f'H100 {gpu_limit:.0f}GB 한계')
ax1.set_xticks(x + w*1.5)
ax1.set_xticklabels([m[0] for m in models])
ax1.set_ylabel('GPU당 메모리 (GB, N=64)')
ax1.set_yscale('log')
ax1.set_title(f'ZeRO Stage별 GPU당 메모리\n(N={N} GPU, H100 80GB)', fontweight='bold')
ax1.legend(fontsize=9); ax1.grid(True, alpha=0.3, axis='y')

# ── 오른쪽: GPU 수에 따른 Stage 3 스케일링 ──────────
ax2 = axes[1]
N_arr = np.array([1, 2, 4, 8, 16, 32, 64, 128, 256])
P_70B = 70

ax2.plot(N_arr, [memory_baseline(P_70B, n) for n in N_arr], 'r-o', lw=2, ms=6, label='Baseline: 1120 GB (상수)')
ax2.plot(N_arr, [memory_zero1(P_70B, n) for n in N_arr], 'orange', marker='s', lw=2, ms=6, label='Stage 1')
ax2.plot(N_arr, [memory_zero2(P_70B, n) for n in N_arr], 'g-^', lw=2, ms=6, label='Stage 2')
ax2.plot(N_arr, [memory_zero3(P_70B, n) for n in N_arr], 'b-D', lw=2, ms=6, label='Stage 3: 16P/N')
ax2.axhline(y=gpu_limit, color='k', ls='--', lw=1.5, alpha=0.7, label=f'H100 {gpu_limit:.0f}GB')

ax2.set_xscale('log', base=2)
ax2.set_xticks(N_arr)
ax2.set_xticklabels([str(n) for n in N_arr], fontsize=9)
ax2.set_xlabel('GPU 수 (N)')
ax2.set_ylabel('GPU당 메모리 (GB)')
ax2.set_title('Llama-2 70B: ZeRO Stage별\nGPU 수 증가에 따른 메모리 절감', fontweight='bold')
ax2.legend(fontsize=9); ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("결론:")
print(f"  - 70B 모델, N=8: Baseline={memory_baseline(70,8):.0f}GB vs Stage3={memory_zero3(70,8):.0f}GB")
print(f"  - Stage 3만이 H100 한계({gpu_limit:.0f}GB) 이하로 들어올 수 있음")

## 7. Stage 선택

In [ ]:
# ---------------------------------------------------
# ZeRO Stage 선택 가이드 및 통신 비용 비교
# ---------------------------------------------------

print("=" * 60)
print("  ZeRO Stage 선택 가이드")
print("=" * 60)
print()
print("┌─────────┬──────────────┬───────────────┬─────────────────┐")
print("│ Stage   │ 통신 횟수    │ 메모리 절약   │ 권장 상황       │")
print("├─────────┼──────────────┼───────────────┼─────────────────┤")
print("│Baseline │ All-Reduce×1 │ 없음          │ 모델이 충분히   │")
print("│         │              │               │ 작을 때         │")
print("├─────────┼──────────────┼───────────────┼─────────────────┤")
print("│Stage 1  │ All-Reduce×1 │ ~50% (N→∞)    │ 메모리 보통     │")
print("│         │ +Scatter×1   │               │ 통신 최소화 원할│")
print("├─────────┼──────────────┼───────────────┼─────────────────┤")
print("│Stage 2  │ Reduce-Scat  │ ~75% (N→∞)    │ 중간 규모       │")
print("│         │ +Gather×1    │               │ 일반 권장       │")
print("├─────────┼──────────────┼───────────────┼─────────────────┤")
print("│Stage 3  │ Reduce-Scat  │ 1/N (선형)    │ 초대형 모델     │")
print("│         │ +Gather×2    │               │ (70B+ 학습)     │")
print("└─────────┴──────────────┴───────────────┴─────────────────┘")

# 실제 추천 시나리오
print("\n=== 모델별 추천 ZeRO Stage ===")
scenarios = [
    (7,   8,   '소형 (7B, N=8)'),
    (70,  16,  '중형 (70B, N=16)'),
    (175, 64,  '대형 (175B, N=64)'),
    (530, 256, '초대형 (530B, N=256)'),
]
for P, N, label in scenarios:
    h100_limit = 80 * 0.7
    mems = [
        memory_baseline(P, N),
        memory_zero1(P, N),
        memory_zero2(P, N),
        memory_zero3(P, N),
    ]
    # 가장 낮은 Stage에서 H100에 들어가는 것 선택
    rec = None
    for s, m in enumerate(mems):
        if m <= h100_limit:
            rec = s
            break
    stage_name = ['Baseline', 'Stage 1', 'Stage 2', 'Stage 3'][rec] if rec is not None else 'Stage 3+Offload'
    print(f"  {label}: 권장={stage_name} ({mems[rec] if rec is not None else mems[3]:.0f}GB/GPU)")

## 8. 정리

### 핵심 개념 요약

| Stage | 분산 대상 | GPU당 메모리 | 추가 통신 |
|-------|----------|------------|----------|
| **0 (Baseline)** | 없음 | $16P$ | All-Reduce 1회 |
| **1** | Optimizer States | $4P + 8P/N$ | Scatter 1회 추가 |
| **2** | + Gradients | $2P + 6P/N$ | Reduce-Scatter로 교체 |
| **3** | + Parameters | $16P/N$ | All-Gather 2회 추가 |

### 핵심 수식

$$M_{ZeRO3} = \frac{16P}{N} \Rightarrow \text{N배 절약, 단 통신 3배}$$

### 다음 챕터 예고
**Chapter 10-04: CPU & NVMe Offloading** — ZeRO Stage 3에서도 모자라면? GPU 메모리를 넘어 CPU RAM과 NVMe SSD로 텐서를 내려보내는 ZeRO-Offload의 원리, PCIe 대역폭 한계와 이를 극복하는 파이프라인 아이디어를 학습한다.